In [2]:
# import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment, ResourceConfiguration
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction
from azure.ai.ml.entities import AmlCompute
from dotenv import load_dotenv, find_dotenv
import os
import mlflow

In [3]:
# Initialize ML Client
load_dotenv(find_dotenv(), override=True)

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

In [11]:
# Create Environment
custom_env_name = "mm-training-env-py37"

try:
    # let's see if the environment already exists
    env = ml_client.environments.get(custom_env_name, label="latest")
    print(f"You already have a environment named {custom_env_name}, we'll reuse it as is.")

except Exception:
    print("Creating a new environment...")
    custom_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for many models parallel training",
        conda_file="../environment/remote_env_conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    )
    custom_job_env = ml_client.environments.create_or_update(custom_job_env)

    print(f"Environment created: {custom_job_env.name}")

You already have a environment named mm-training-env-py37, we'll reuse it as is.


In [12]:
# Acess data asset
data_name = "oj-sim-sales-train"
train_data_asset = ml_client.data.get(data_name, label='latest')
print(train_data_asset.path)

azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/eabbea2d8b14c97a278f2f297b333403/train_subset.csv


In [4]:
# Load data preparation component
partition_data = load_component(source="../src/components/partition_data/partition_data.yaml")

In [6]:
#Register customer data partition component
ml_client.components.create_or_update(partition_data)

CommandComponent({'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'partition_data', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/components/partition_data/versions/1', 'Resource__source_path': None, 'base_path': '/home/zacksoenen/Projects/many-models-azureml/demo_notebooks', 'creation_context': <azure.ai.ml._restclient.v2022_10_01.models._models_py3.SystemData object at 0x7f032d31edc0>, 'serialize': <msrest.serialization.Serializer object at 0x7f032d331760>, 'command': 'python partition_data.py --data_source ${{inputs.data_source}} --partition_keys ${{inputs.partition_keys}} --tabular_output_data ${{outputs.tabular_output_data}}', 'code': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d

In [14]:
# Load Azure pre-build component
'''ml_client_registry = MLClient(credential=credential, registry_name="azureml")
register_model_from_registry = ml_client_registry.components.get(
    name="register_model", label="latest"
)'''

In [18]:
# Declare parallel job, with a run_function task
many_model_training_with_partition_keys = parallel_run_function(
    name="train_many_models_with_partition_keys",
    display_name="Train Many Models With Partition Keys",
    description="parallel job to train many models with partition_keys on mltable input",
    inputs=dict(
        data_source=Input(
            type=AssetTypes.MLTABLE,
            description="Input mltable with predefined partition format.",
            mode=InputOutputModes.DIRECT,  # [Important] To use 'partition_keys', input MLTable is required to use 'direct' mode.
        ),
        drop_cols=Input(
            type="string",
            description="Columns need to be dropped before training. Split by comma.",
        ),
        target_col=Input(
            type="string",
            description="The column name for label of the input data.",
        ),
        date_col=Input(
            type="string",
            description="The column name for datatime. This will be used for generating time-series lagging data.",
        ),
        lagging_orders=Input(
            type="string",
            description="List of int which indicate how to generate lagging data for time-series input. Split by comma.",
        ),
    ),
    outputs=dict(
        model_folder=Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
        )
    ),
    input_data="${{inputs.data_source}}",  # Define which input data will be splitted into mini-batches
    partition_keys=["Store","Brand"],  # Use 'partition_keys' as the data division method. This method requires MLTable input with partition setting pre-defined in MLTable artifact.
    instance_count=6,  # Use X nodes from compute cluster to run this parallel job.
    max_concurrency_per_instance=2,  # Create 2 worker processors in each compute node to execute mini-batches.
    error_threshold=-1,  # Monitor the failures of item processed by the gap between mini-batch input count and returns. 'Many model training' scenario doesn't fit this setting and '-1' means ignore counting failure items by mini-batch returns.
    mini_batch_error_threshold=5,  # Monitor the failed mini-batch by exception, time out, or null return. When failed mini-batch count is higher than this setting, the parallel job will be marked as 'failed'.
    retry_settings=dict(
        max_retries=1,  # Define how many retries when mini-batch execution is failed by exception, time out, or null return.
        timeout=60,  # Define the timeout in second for each mini-batch execution.
    ),
    logging_level="DEBUG", # DEBUG, INFO, WARNING, ERROR, ETC
    environment_variables={
        "AZUREML_PARALLEL_EXAMPLE": "1a_sdk",
    },
    task=RunFunction(
        code="../src/",
        entry_script="parallel_train.py",
        environment=ml_client.environments.get(custom_env_name, label="latest"),
        program_arguments="--drop_cols ${{inputs.drop_cols}} "  # Passthrough input parameters into parallel_train script.
        "--target_col ${{inputs.target_col}} "
        "--date_col ${{inputs.date_col}} "
        "--lagging_orders ${{inputs.lagging_orders}} "
        "--model_folder ${{outputs.model_folder}} ",
    ),
)

In [19]:
# Build Pipeline
# Declare the overall input of the job.
input_oj_data = Input(
    path=train_data_asset.path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.RO_MOUNT,
)

# Declare pipeline structure.
@pipeline(display_name="parallel job for oj many model training")
def parallel_training_pipeline(pipeline_input_data):
    
    # Declare 1st data partition command job.
    partition_job = partition_data(
        data_source=pipeline_input_data,
        partition_keys="Store,Brand",
    )

    # Declare 2nd parallel model training job.
    parallel_train = many_model_training_with_partition_keys(
        data_source=partition_job.outputs.tabular_output_data,
        drop_cols="Advert,Store,Brand",
        target_col="Revenue",
        date_col="WeekStarting",
        lagging_orders="1,2,3,4,5,6",
    )
    
    # User could override parallel job run-level property when invoke that parallel job/component in pipeline.
    # parallel_train.resources.instance_count = 5
    # parallel_train.max_concurrency_per_instance = 2
    # parallel_train.mini_batch_error_threshold = 10

# Create pipeline instance
training_pipeline = parallel_training_pipeline(pipeline_input_data=input_oj_data,)

# Set pipeline level compute
training_pipeline.settings.default_compute = cpu_compute_target
print(training_pipeline)

display_name: parallel job for oj many model training
type: pipeline
inputs:
  pipeline_input_data:
    mode: ro_mount
    type: uri_file
    path: azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/eabbea2d8b14c97a278f2f297b333403/train_subset.csv
jobs:
  partition_job:
    type: command
    inputs:
      data_source:
        path: ${{parent.inputs.pipeline_input_data}}
      partition_keys: Store,Brand
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: partition_data
      version: '1'
      display_name: Partition input data by keys
      type: command
      inputs:
        data_source:
          type: uri_file
        partition_keys:
          type: string
      outputs:
        tabular_output_data:
          type: mltable
      command: python partition_data.py --data_source ${{inputs.data_source}} --partitio

In [20]:
# Submit training pipeline
pipeline_job = ml_client.jobs.create_or_update(
    training_pipeline,
    experiment_name="many-models-parallel-training-job-mlflow24",
)


Uploading src (0.01 MBs): 100%|██████████| 8808/8808 [00:00<00:00, 40411.37it/s]




In [47]:
# Analyze results using mlflow

azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_tracking_uri)

EXPIREMENTS = ["many-models-parallel-training-job-mlflow24"]

mlflow.search_runs(experiment_names= EXPIREMENTS,
                   filter_string="tags.store = '1013' and \
                                  metrics.test_mape < 30 and \
                                  tags.brand = 'tropicana' "
                )

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_mse,metrics.training_score,metrics.training_mae,metrics.training_rmse,...,params.min_samples_leaf,params.max_leaf_nodes,params.min_samples_split,tags.store,tags.mlflow.runName,tags.estimator_name,tags.brand,tags.mlflow.rootRunId,tags.mlflow.user,tags.estimator_class
0,8e65bc4f-d3a9-4dce-9093-49c0e9fc3afd,e7fcc503-1735-43d4-9298-1228018a7232,FINISHED,,2023-12-21 22:15:01.902000+00:00,2023-12-21 22:15:11.010000+00:00,1.534044e+06,0.979091,989.14651,1238.565122,...,1,None,2,1013,train_1013_tropicana,GradientBoostingRegressor,tropicana,8e65bc4f-d3a9-4dce-9093-49c0e9fc3afd,Zack Soenen,sklearn.ensemble.gradient_boosting.GradientBoo...


In [70]:
# Get best version of the registered model
df = mlflow.search_runs(experiment_names= EXPIREMENTS,
                   filter_string="tags.store = '1013' and \
                                  tags.brand = 'tropicana' "
                )

best_run = df.loc[df['metrics.test_rmse'].idxmin()]['run_id']

child_job_df = mlflow.search_runs(experiment_names= EXPIREMENTS, filter_string="tags.mlflow.parentRunId = '8e65bc4f-d3a9-4dce-9093-49c0e9fc3afd'")
best_run = child_job_df.loc[child_job_df['tags.mlflow.parentRunId']== best_run, 'run_id'].values[0]

for model in ml_client.models.list(name="1013_tropicana"):
    if model.job_name == best_run:
        print(f'BEST MODEL VERSION: {model.version}')

BEST MODEL VERSION: 13
